In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
import pandas_datareader as pdr



In [28]:
df=pd.read_csv('gbp_test.csv')
df.drop(df.tail(1).index,
        inplace = True)
df

Unnamed: 0     指标名称  GBP_USD  美元指数:月:平均值  美国:CPI:当月同比  \
0           0.0  2011/01   1.6015   79.136667          1.6   
1           1.0  2011/02   1.6258   77.762000          2.1   
2           2.0  2011/03   1.6031   76.260000          2.7   
3           3.0  2011/04   1.6701   74.691429          3.2   
4           4.0  2011/05   1.6449   74.933182          3.6   
..          ...      ...      ...         ...          ...   
132       132.0  2022/01   1.3445   95.938267          7.5   
133       133.0  2022/02   1.3419   96.005875          7.9   
134       134.0  2022/03   1.3133   98.463757          8.5   
135       135.0  2022/04   1.2571  100.689371          8.3   
136       136.0  2022/05   1.2600  103.084923          8.6   

     美国:供应管理协会(ISM):制造业PMI  期货结算价(连续):WTI原油:月:平均值  期货收盘价(连续):COMEX黄金:月:平均值  \
0                     59.0              89.578500                 7.216181   
1                     59.3              89.743158                 7.225236   
2                     59.1             102.981304                 7.260455   
3                     58.9             110.038500                 7.302015   
4                     53.7             101.356667                 7.320177   
..                     ...                    ...                      ...   
132                   57.6              82.977143                 7.504297   
133                   58.6              91.605000                 7.527349   
134                   57.1             108.153913                 7.578901   
135                   55.4             101.612500                 7.567294   
136                   56.1             109.470909                 7.518163   

     美国:道琼斯工业平均指数:月:平均值  美国:纳斯达克综合指数:月:平均值     美国:M2  美国:标准普尔500指数:月:平均值  \
0              9.376056           7.907360  9.087868            7.156659   
1              9.408371           7.931477  9.092052            7.186234   
2              9.399429           7.909228  9.104724            7.173564   
3              9.428260           7.936328  9.110829            7.194065   
4              9.439863           7.942744  9.107521            7.199163   
..                  ...                ...       ...                 ...   
132           10.476052           9.584066  9.982599            8.428103   
133           10.453009           9.539553  9.981924            8.397504   
134           10.434990           9.519534  9.995406            8.387373   
135           10.443338           9.502574  9.995871            8.387380   
136           10.385280           9.378371  9.983204            8.304089   

     美国:标准普尔500波动率指数(VIX):月:平均值  美国:密歇根大学消费者信心指数  全球:经济政策不确定性指数:按GDP现价加权平均  \
0                     17.315500             74.2                  4.702427   
1                     17.430000             77.5                  4.534989   
2                     20.723478             67.5                  4.836210   
3                     16.244000             69.8                  4.742122   
4                     16.911429             74.3                  4.485622   
..                          ...              ...                       ...   
132                   23.181000             67.2                  5.449804   
133                   25.748421             62.8                  5.243335   
134                   26.968696             59.4                  5.801087   
135                   24.373500             65.2                  5.675489   
136                   29.445714             58.4                  5.676240   

     美国:经济政策不确定性指数:总指数  美国:经济政策不确定性指数:新闻指数     CLI_US  CLI_global  
0             4.957027            4.713960  100.25530   101.01810  
1             4.980237            4.649784  100.34860   101.03670  
2             5.161090            5.035924  100.33740   100.97800  
3             5.059399            4.827086  100.22080   100.84340  
4             4.837805            4.437333  100.01280   100.64510  
..                 ...                 ...        ...         ...

# No LASSO models

In [29]:
import pandas as pd
import numpy as np
df=pd.read_csv('gbp_test.csv')
df.drop(df.tail(1).index,
        inplace = True)

df = df.drop('Unnamed: 0',axis=1)
df = df.drop('指标名称',axis=1)
import warnings
warnings.filterwarnings('ignore')

all_data = np.array(df)

# Train set test set split
row = 0.2*(all_data.shape[0])
x_train_initial = all_data[:-int(row), 1:]
x_test_initial = all_data[-int(row):, 1:]

y_train_initial = all_data[:-int(row), 0]
y_test_initial = all_data[-int(row):, 0]

# Standard Scaler
from sklearn.preprocessing import StandardScaler,MinMaxScaler
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

x_scaled = x_scaler.fit(x_train_initial)
x_train = x_scaler.transform(x_train_initial)
x_test = x_scaler.transform(x_test_initial)

y_scaler = y_scaler.fit(y_train_initial.reshape(-1,1))
y_train = y_scaler.transform(y_train_initial.reshape(-1,1))
y_test = y_scaler.transform(y_test_initial.reshape(-1,1))

### MODELS
#Model 1: SVR
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
svr = SVR()
svr.fit(x_train, y_train.ravel())

y_pre = svr.predict(x_test)
y_pre = y_pre.reshape(-1,1)

y_test_pre = y_scaler.inverse_transform(y_pre)
y_test_pre = y_test_pre.reshape(-1,1)
y_test_rel = y_test_initial

y_svr_pre = y_test_pre
from accuracy import RMSE,MAE,MAPE
mae_svr = MAE(y_test_rel,y_svr_pre)
rmse_svr = RMSE(y_test_rel,y_svr_pre)
mape_svr = MAPE(y_test_rel,y_svr_pre)
print('svr的MAE为：'+str(mae_svr))
print('svr的RMSE为：'+str(rmse_svr)) 

#Model 2: Extreme Learning Machinge (ELM)
#A. Akusok, K. -M. Björk, Y. Miche and A. Lendasse, "High-Performance Extreme Learning Machines: A Complete Toolbox for Big Data 
#Applications," in IEEE Access, vol. 3, pp. 1011-1025, 2015, doi: 10.1109/ACCESS.2015.2450498.
from hpelm import ELM

elm=ELM(16, 1)
elm.add_neurons(1, "sigm")
elm.add_neurons(1, "rbf_l2")
elm.train(x_train, y_train, "LOO")
y_pre = elm.predict(x_test)


y_pre = y_pre.reshape(-1,1)
y_test_pre = y_scaler.inverse_transform(y_pre)
y_test_pre = y_test_pre.reshape(-1,1)

y_elm_pre = y_test_pre
mae_elm = MAE(y_test_rel,y_elm_pre)
rmse_elm = RMSE(y_test_rel,y_elm_pre)
mape_elm = MAPE(y_test_rel,y_elm_pre)
print('elm的MAE为：'+str(mae_elm))
print('elm的RMSE为：'+str(rmse_elm))

#Model 3: LSTM
import os
import random
import tensorflow as tf
np.random.seed(1000)
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1000)
tf.random.set_seed(1000)

#Reshape our model
amount_of_features = 1
x_train1 = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
x_test1 = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))

from keras.layers import LSTM,Dense,Dropout
from keras.models import Sequential
verbose = 0
model = Sequential()
model.add(LSTM(12,input_shape=(16,1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

model.fit(x_train1,y_train,epochs=100,validation_split=0,verbose=0)
y_pre = model.predict(x_test1)

y_pre = y_pre.reshape(-1,1)
y_test_pre = y_scaler.inverse_transform(y_pre)
y_test_pre = y_test_pre.reshape(-1,1)

y_lstm_pre = y_test_pre
mae_lstm = MAE(y_test_rel,y_lstm_pre)
rmse_lstm = RMSE(y_test_rel,y_lstm_pre)
mape_lstm = MAPE(y_test_rel,y_lstm_pre)
print('LSTM的MAE为：'+str(mae_lstm)) 
print('LSTM的RMSE为：'+str(rmse_lstm)) 

#Model 4: Bidirectional LSTM
#Reshape our model
amount_of_features = 1
x_train1 = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
x_test1 = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))

from keras.layers import Bidirectional
verbose = 0
model = Sequential()
model.add(Bidirectional(LSTM(16,input_shape=(16,1))))
model.add(Dropout(0.05))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

model.fit(x_train1,y_train,epochs=100+100,validation_split=0,verbose=0)
y_pre = model.predict(x_test1)

y_pre = y_pre.reshape(-1,1)
y_test_pre = y_scaler.inverse_transform(y_pre)
y_test_pre = y_test_pre.reshape(-1,1)

y_bilstm_pre = y_test_pre
mae_bilstm = MAE(y_test_rel,y_bilstm_pre)
rmse_bilstm = RMSE(y_test_rel,y_bilstm_pre)
mape_bilstm = MAPE(y_test_rel,y_bilstm_pre)
print('BiLSTM的MAE为：'+str(mae_bilstm)) 
print('BiLSTM的RMSE为：'+str(rmse_bilstm)) 

#Model 5: Kernel ELM
#jlsuarezdiaz. (2020). KELMOR. Github. Retrieved November 22, 2022, from https://github.com/jlsuarezdiaz/KELMOR.
from sys import path
path.append(".")

kelmor_module = __import__("KELMOR") 
KELMOR = kelmor_module.KELMOR

kelmor = KELMOR(C=2, kernel="rbf")
kelmor.fit(x_train,y_train)

y_pre = kelmor.predict(x_test)

y_pre = y_pre.reshape(-1,1)

y_test_pre = y_scaler.inverse_transform(y_pre)
y_test_pre = y_test_pre.reshape(-1,1)
y_test_rel = y_test_initial

y_kelm_pre = y_test_pre
from accuracy import RMSE,MAE,MAPE
mae_kelm = MAE(y_test_rel,y_kelm_pre)
rmse_kelm = RMSE(y_test_rel,y_kelm_pre)
mape_kelm = MAPE(y_test_rel,y_kelm_pre)
print('KELM的MAE为：'+str(mae_kelm))
print('KELM的RMSE为：'+str(rmse_kelm))
svr1=y_svr_pre
elm1=y_elm_pre
kelm1=y_kelm_pre
lstm1=y_lstm_pre
bilstm1=y_bilstm_pre

svr的MAE为：0.13545200337357544
svr的RMSE为：0.1511313647427158
elm的MAE为：0.07825458341793547
elm的RMSE为：0.10014639454597614
1/1 [==============================] - 0s 236ms/step
LSTM的MAE为：0.1450084121633459
LSTM的RMSE为：0.15543700890824597
1/1 [==============================] - 0s 441ms/step
BiLSTM的MAE为：0.04131451497960973
BiLSTM的RMSE为：0.04536984682997785
KELM的MAE为：0.06464814814814814
KELM的RMSE为：0.07678033843615414


# LASSO tests

In [11]:
LABEL = 'GBP_USD' # response variable
FEATURES = ('美元指数:月:平均值','期货结算价(连续):WTI原油:月:平均值','期货收盘价(连续):COMEX黄金:月:平均值','美国:道琼斯工业平均指数:月:平均值','美国:纳斯达克综合指数:月:平均值','美国:M2','美国:标准普尔500指数:月:平均值','美国:标准普尔500波动率指数(VIX):月:平均值','美国:密歇根大学消费者信心指数','全球:经济政策不确定性指数:按GDP现价加权平均','美国:经济政策不确定性指数:总指数','美国:经济政策不确定性指数:新闻指数', 'CLI_US', 'CLI_global') # explanatory variables
FEATURES


('美元指数:月:平均值',
 '期货结算价(连续):WTI原油:月:平均值',
 '期货收盘价(连续):COMEX黄金:月:平均值',
 '美国:道琼斯工业平均指数:月:平均值',
 '美国:纳斯达克综合指数:月:平均值',
 '美国:M2',
 '美国:标准普尔500指数:月:平均值',
 '美国:标准普尔500波动率指数(VIX):月:平均值',
 '美国:密歇根大学消费者信心指数',
 '全球:经济政策不确定性指数:按GDP现价加权平均',
 '美国:经济政策不确定性指数:总指数',
 '美国:经济政策不确定性指数:新闻指数',
 'CLI_US',
 'CLI_global')

In [13]:
df_new=df.loc[:, FEATURES]
response=df.loc[:,LABEL]
df_new
len(FEATURES)

14

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, FEATURES], df.loc[:, LABEL], shuffle=False, test_size=0.2)
X_test

美元指数:月:平均值  期货结算价(连续):WTI原油:月:平均值  期货收盘价(连续):COMEX黄金:月:平均值  \
109   98.877460              50.635500                 7.374764   
110   98.789655              30.454545                 7.372800   
111   99.866591              16.699048                 7.435387   
112   99.587319              28.749048                 7.449085   
113   97.096186              38.313636                 7.459475   
114   95.592983              40.743043                 7.521200   
115   93.033129              42.399524                 7.584652   
116   93.408764              39.610000                 7.561418   
117   93.405914              39.554545                 7.548513   
118   92.604662              41.530000                 7.531217   
119   90.480213              47.078636                 7.528642   
120   90.241765              52.103000                 7.532171   
121   90.629355              59.125500                 7.498450   
122   92.035513              62.357826                 7.450986   
123   91.638386              61.714762                 7.472604   
124   90.306252              65.234762                 7.522746   
125   91.057341              71.343182                 7.514924   
126   92.510541              72.598182                 7.499061   
127   92.806050              67.703182                 7.486736   
128   92.975636              71.422727                 7.480394   
129   93.960176              81.201429                 7.482175   
130   95.326768              78.621364                 7.506987   
131   96.207952              71.767391                 7.491273   
132   95.938267              82.977143                 7.504297   
133   96.005875              91.605000                 7.527349   
134   98.463757             108.153913                 7.578901   
135  100.689371             101.612500                 7.567294   
136  103.084923             109.470909                 7.518163   

     美国:道琼斯工业平均指数:月:平均值  美国:纳斯达克综合指数:月:平均值     美国:M2  美国:标准普尔500指数:月:平均值  \
109           10.258351           9.150468  9.641025            8.094780   
110           10.027360           8.958309  9.683844            7.883218   
111           10.055947           9.023096  9.746974            7.923701   
112           10.097038           9.116651  9.785092            7.979207   
113           10.168244           9.194208  9.803872            8.040660   
114           10.180582           9.259117  9.812074            8.073284   
115           10.233560           9.324766  9.815705            8.129089   
116           10.230393           9.313680  9.827853            8.121337   
117           10.240142           9.344455  9.835696            8.137016   
118           10.279319           9.375385  9.851173            8.174419   
119           10.313893           9.442963  9.865157            8.214820   
120           10.335963           9.486957  9.871584            8.241110   
121           10.350859           9.527878  9.879400            8.264475   
122           10.385089           9.485607  9.900789            8.271423   
123           10.428313           9.539676  9.916887            8.328735   
124           10.442035           9.511942  9.919282            8.335155   
125           10.442606           9.553729  9.925229            8.351962   
126           10.457338           9.591449  9.931638            8.381079   
127           10.470050           9.606194  9.941602            8.401604   
128           10.454161           9.617373  9.950223            8.399657   
129           10.464688           9.607707  9.956843            8.403063   
130           10.487059           9.668710  9.967799            8.448355   
131           10.481261           9.646944  9.982922            8.449936   
132           10.476052           9.584066  9.982599            8.428103   
133           10.453009           9.539553  9.981924            8.397504   
134           10.434990           9.519534  9.995406            8.387373   
135      

In [15]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model' ,Lasso())
])

search=GridSearchCV(pipeline,
                               {'model__alpha':np.arange(0,10,0.04)},
                               cv=5,
                               scoring='neg_mean_squared_error',
                               verbose=3
                   
                               )

search.fit(X_train,y_train)
search.best_params_
coef=search.best_estimator_[1].coef_
a=np.array(FEATURES)[coef != 0]
a

Fitting 5 folds for each of 250 candidates, totalling 1250 fits
[CV 1/5] END .................model__alpha=0.0;, score=-0.016 total time=   0.0s
[CV 2/5] END .................model__alpha=0.0;, score=-0.003 total time=   0.0s
[CV 3/5] END .................model__alpha=0.0;, score=-0.031 total time=   0.0s
[CV 4/5] END .................model__alpha=0.0;, score=-0.007 total time=   0.0s
[CV 5/5] END .................model__alpha=0.0;, score=-0.019 total time=   0.0s
[CV 1/5] END ................model__alpha=0.04;, score=-0.001 total time=   0.0s
[CV 2/5] END ................model__alpha=0.04;, score=-0.018 total time=   0.0s
[CV 3/5] END ................model__alpha=0.04;, score=-0.026 total time=   0.0s
[CV 4/5] END ................model__alpha=0.04;, score=-0.018 total time=   0.0s
[CV 5/5] END ................model__alpha=0.04;, score=-0.010 total time=   0.0s
[CV 1/5] END ................model__alpha=0.08;, score=-0.005 total time=   0.0s
[CV 2/5] END ................model__alpha=0.0

[CV 1/5] END ................model__alpha=1.04;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=1.04;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=1.04;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=1.04;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=1.04;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=1.08;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=1.08;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=1.08;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=1.08;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=1.08;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=1.12;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=1.12;, score=-0.037 total time=   0.0s
[CV 3/5] END ...............

[CV 2/5] END ..model__alpha=1.8800000000000001;, score=-0.037 total time=   0.0s
[CV 3/5] END ..model__alpha=1.8800000000000001;, score=-0.007 total time=   0.0s
[CV 4/5] END ..model__alpha=1.8800000000000001;, score=-0.040 total time=   0.0s
[CV 5/5] END ..model__alpha=1.8800000000000001;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=1.92;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=1.92;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=1.92;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=1.92;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=1.92;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=1.96;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=1.96;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=1.96;, score=-0.007 total time=   0.0s
[CV 4/5] END ...............

[CV 1/5] END ..model__alpha=2.8000000000000003;, score=-0.030 total time=   0.0s
[CV 2/5] END ..model__alpha=2.8000000000000003;, score=-0.037 total time=   0.0s
[CV 3/5] END ..model__alpha=2.8000000000000003;, score=-0.007 total time=   0.0s
[CV 4/5] END ..model__alpha=2.8000000000000003;, score=-0.040 total time=   0.0s
[CV 5/5] END ..model__alpha=2.8000000000000003;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=2.84;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=2.84;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=2.84;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=2.84;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=2.84;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=2.88;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=2.88;, score=-0.037 total time=   0.0s
[CV 3/5] END ...............

[CV 4/5] END ................model__alpha=3.64;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=3.64;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=3.68;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=3.68;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=3.68;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=3.68;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=3.68;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=3.72;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=3.72;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=3.72;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=3.72;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=3.72;, score=-0.049 total time=   0.0s
[CV 1/5] END ..model__alpha=

...

[CV 4/5] END ................model__alpha=7.08;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=7.08;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=7.12;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=7.12;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=7.12;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=7.12;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=7.12;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=7.16;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=7.16;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=7.16;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=7.16;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=7.16;, score=-0.049 total time=   0.0s
[CV 1/5] END ...............

[CV 4/5] END ................model__alpha=7.92;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=7.92;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=7.96;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=7.96;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=7.96;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=7.96;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=7.96;, score=-0.049 total time=   0.0s
[CV 1/5] END .................model__alpha=8.0;, score=-0.030 total time=   0.0s
[CV 2/5] END .................model__alpha=8.0;, score=-0.037 total time=   0.0s
[CV 3/5] END .................model__alpha=8.0;, score=-0.007 total time=   0.0s
[CV 4/5] END .................model__alpha=8.0;, score=-0.040 total time=   0.0s
[CV 5/5] END .................model__alpha=8.0;, score=-0.049 total time=   0.0s
[CV 1/5] END ...model__alpha

[CV 2/5] END .................model__alpha=8.8;, score=-0.037 total time=   0.0s
[CV 3/5] END .................model__alpha=8.8;, score=-0.007 total time=   0.0s
[CV 4/5] END .................model__alpha=8.8;, score=-0.040 total time=   0.0s
[CV 5/5] END .................model__alpha=8.8;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=8.84;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=8.84;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=8.84;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=8.84;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=8.84;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=8.88;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=8.88;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=8.88;, score=-0.007 total time=   0.0s
[CV 4/5] END ...............

[CV 2/5] END ................model__alpha=9.68;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=9.68;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=9.68;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=9.68;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=9.72;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=9.72;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=9.72;, score=-0.007 total time=   0.0s
[CV 4/5] END ................model__alpha=9.72;, score=-0.040 total time=   0.0s
[CV 5/5] END ................model__alpha=9.72;, score=-0.049 total time=   0.0s
[CV 1/5] END ................model__alpha=9.76;, score=-0.030 total time=   0.0s
[CV 2/5] END ................model__alpha=9.76;, score=-0.037 total time=   0.0s
[CV 3/5] END ................model__alpha=9.76;, score=-0.007 total time=   0.0s
[CV 4/5] END ...............

array(['美元指数:月:平均值', '美国:M2', '全球:经济政策不确定性指数:按GDP现价加权平均'], dtype='<U26')

In [20]:
LF=['GBP_USD','美元指数:月:平均值', '美国:M2', '全球:经济政策不确定性指数:按GDP现价加权平均']

In [27]:
import pandas as pd
import numpy as np
df=df=pd.read_csv('gbp_test.csv')
df.drop(df.tail(1).index,
        inplace = True)
df = df.loc[:, LF]


import warnings
warnings.filterwarnings('ignore')

#Change data shape
sequence_length = 6
horizon = 1
data = np.array(df.iloc[:,0]).reshape([-1,1])

all_data = []
for dta in range(len(data) - sequence_length-horizon+1):
    all_data.append(data[dta: dta + sequence_length])

all_data = np.array(all_data)
all_data = all_data.reshape(all_data.shape[0],all_data.shape[1])
ydata = data[horizon+sequence_length-1:]
ydata = pd.DataFrame(list(ydata))

#ydata = list(ydata)
all_data = pd.DataFrame(all_data)
all_data["y"] = ydata
all_data = np.array(all_data)

#train test split
row = round(0.2 * int(data.shape[0]))
x_train_initial = all_data[:-int(row), :-1]
x_test_initial = all_data[-int(row):, :-1]
y_train_initial = all_data[:-int(row), -1]
y_test_initial = all_data[-int(row):, -1]


#Scaler
from sklearn.preprocessing import StandardScaler,MinMaxScaler
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

x_scaled = x_scaler.fit(x_train_initial)
x_train = x_scaler.transform(x_train_initial)
x_test = x_scaler.transform(x_test_initial)

y_scaler = y_scaler.fit(y_train_initial.reshape(-1,1))
y_train = y_scaler.transform(y_train_initial.reshape(-1,1))
y_test = y_scaler.transform(y_test_initial.reshape(-1,1))

# MODELS
#Model 1: SVR
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
svr = SVR()
svr.fit(x_train, y_train.ravel())

y_pre = svr.predict(x_test)
y_pre = y_pre.reshape(-1,1)

y_test_pre = y_scaler.inverse_transform(y_pre)
y_test_pre = y_test_pre.reshape(-1,1)
y_test_rel = y_test_initial

y_svr_pre = y_test_pre
from accuracy import RMSE,MAE,MAPE
mae_svr = MAE(y_test_rel,y_svr_pre)
rmse_svr = RMSE(y_test_rel,y_svr_pre)
mape_svr = MAPE(y_test_rel,y_svr_pre)
print('svr的MAE为：'+str(mae_svr)) #"MAE:"
print('svr的RMSE为：'+str(rmse_svr)) #"RMSE:"


#MODEL 2: ELM
from hpelm import ELM

elm=ELM(6, 1)
elm.add_neurons(1, "sigm")
elm.add_neurons(1, "rbf_l2")
elm.train(x_train, y_train, "LOO")
y_pre = elm.predict(x_test)


y_pre = y_pre.reshape(-1,1)
y_test_pre = y_scaler.inverse_transform(y_pre)
y_test_pre = y_test_pre.reshape(-1,1)

y_elm_pre = y_test_pre
mae_elm = MAE(y_test_rel,y_elm_pre)
rmse_elm = RMSE(y_test_rel,y_elm_pre)
mape_elm = MAPE(y_test_rel,y_elm_pre)
print('elm的MAE为：'+str(mae_elm)) 
print('elm的RMSE为：'+str(rmse_elm)) 

#Model 3: LSTM

import os
import random
import tensorflow as tf

np.random.seed(1000)

os.environ['PYTHONHASHSEED'] = '0'
random.seed(1000)
tf.random.set_seed(1000)

amount_of_features = 1
x_train1 = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
x_test1 = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))
#
from keras.layers import LSTM,Dense,Dropout
from keras.models import Sequential
verbose = 0
model = Sequential()
model.add(LSTM(12,input_shape=(6,1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

model.fit(x_train1,y_train,epochs=100,validation_split=0,verbose=0)
y_pre = model.predict(x_test1)

y_pre = y_pre.reshape(-1,1)
y_test_pre = y_scaler.inverse_transform(y_pre)
y_test_pre = y_test_pre.reshape(-1,1)

y_lstm_pre = y_test_pre
mae_lstm = MAE(y_test_rel,y_lstm_pre)
rmse_lstm = RMSE(y_test_rel,y_lstm_pre)
mape_lstm = MAPE(y_test_rel,y_lstm_pre)
print('LSTM的MAE为：'+str(mae_lstm)) #"MAE:"
print('LSTM的RMSE为：'+str(rmse_lstm)) #"RMSE:"


#Model 4: BiLSTM
amount_of_features = 1
x_train1 = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
x_test1 = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))
#
from keras.layers import Bidirectional
verbose = 0
model = Sequential()
model.add(Bidirectional(LSTM(16,input_shape=(6,1))))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

model.fit(x_train1,y_train,epochs=400+400,validation_split=0,verbose=0)
y_pre = model.predict(x_test1)

y_pre = y_pre.reshape(-1,1)
y_test_pre = y_scaler.inverse_transform(y_pre)
y_test_pre = y_test_pre.reshape(-1,1)

y_bilstm_pre = y_test_pre
mae_bilstm = MAE(y_test_rel,y_bilstm_pre)
rmse_bilstm = RMSE(y_test_rel,y_bilstm_pre)
mape_bilstm = MAPE(y_test_rel,y_bilstm_pre)
print('BiLSTM的MAE为：'+str(mae_bilstm)) #"MAE:"
print('BiLSTM的RMSE为：'+str(rmse_bilstm)) #"RMSE:"

# Model 5: KELM
from sys import path

path.append(".") 

kelmor_module = __import__("KELMOR") 
KELMOR = kelmor_module.KELMOR

kelmor = KELMOR(C=2, kernel="rbf")
kelmor.fit(x_train,y_train)

y_pre = kelmor.predict(x_test)

y_pre = y_pre.reshape(-1,1)

y_test_pre = y_scaler.inverse_transform(y_pre)
y_test_pre = y_test_pre.reshape(-1,1)
y_test_rel = y_test_initial

y_kelm_pre = y_test_pre
from accuracy import RMSE,MAE,MAPE
mae_kelm = MAE(y_test_rel,y_kelm_pre)
rmse_kelm = RMSE(y_test_rel,y_kelm_pre)
mape_kelm = MAPE(y_test_rel,y_kelm_pre)
print('KELM的MAE为：'+str(mae_kelm)) #"MAE:"
print('KELM的RMSE为：'+str(rmse_kelm)) #"RMSE:"

svr的MAE为：0.034004797682276905
svr的RMSE为：0.038562451737025626
elm的MAE为：0.06386827465277108
elm的RMSE为：0.07850781868736863
1/1 [==============================] - 0s 280ms/step
LSTM的MAE为：0.03539160285525852
LSTM的RMSE为：0.04293108059624453
1/1 [==============================] - 0s 496ms/step
BiLSTM的MAE为：0.02533941693482576
BiLSTM的RMSE为：0.030481333783907293
KELM的MAE为：0.0439296296296296
KELM的RMSE为：0.04963339303033495
